In [1]:
"""
SHERPA is a Python library for hyperparameter tuning of machine learning models.
Copyright (C) 2018  Lars Hertel, Peter Sadowski, and Julian Collado.

This file is part of SHERPA.

SHERPA is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

SHERPA is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with SHERPA.  If not, see <http://www.gnu.org/licenses/>.
"""
from __future__ import print_function
import sherpa
import sherpa.algorithms.bayesian_optimization as bayesian_optimization
import time
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.datasets import mnist

Using TensorFlow backend.


In [2]:
batch_size = 32
num_classes = 10
epochs = 15

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

60000 train samples
10000 test samples


In [4]:
parameters = [sherpa.Discrete('num_units', [32, 128]),
              sherpa.Choice('activation', ['relu', 'tanh', 'sigmoid'])]

In [5]:
algorithm = bayesian_optimization.GPyOpt(max_concurrent=1,
                                           model_type='GP_MCMC',
                                           acquisition_type='EI_MCMC')

In [6]:
study = sherpa.Study(parameters=parameters,
                     algorithm=algorithm,
                     lower_is_better=True)

INFO:sherpa.core:
-------------------------------------------------------
SHERPA Dashboard running. Access via
http://192.168.2.4:8880 if on a cluster or
http://localhost:8880 if running locally.
-------------------------------------------------------


In [ ]:
num_iterations = 10

for trial in study:
    print("Trial {}:\t{}".format(trial.id, trial.parameters))

    model = Sequential()
    model.add(Dense(units=trial.parameters['num_units'], activation=trial.parameters['activation'], input_dim=784))
    model.add(Dense(units=10, activation='softmax'))
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              callbacks=[study.keras_callback(trial, objective_name='val_loss')],
              validation_data=(x_test, y_test))

    study.finalize(trial=trial)

In [ ]:
print(study.get_best_result())